# LangChain

LangChain is a powerful library designed to simplify the creation of applications powered by Large Language Models (LLMs). It enables developers to connect language models (like GPT) with external data sources (APIs, databases, or knowledge bases), perform chains of tasks, and integrate different tools to create end-to-end workflows.


**[Langchain Crash Course](https://www.youtube.com/watch?v=yF9kGESAi3M)**


## Core Components of LangChain
- 1. LLMs
- 2. Prompts
- 3. Chains
- 4. RAGs
- 5. Agents
- 6. Tools 
- 7. Memory 


*Let's setup a development environment*

In [ ]:
!pip install python-dotenv


In [152]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

## Chat models (LLMs)

A chat model in LangChain is a Component designed to communicate in a sytructured way with LLMs like GPT-4, Huggingface,and Etc.
https://python.langchain.com/docs/integrations/chat/

i.e. _Hugging Face offers thousands of pretrained models available through their Model Hub. These models can be fine-tuned for specific tasks, saving time and computational resources._
_It is widely recognized for its open-source machine learning models, tools, and datasets, which are designed to make it easier for researchers and developers to build, train, and deploy AI models._


In [ ]:
!pip install -qU langchain-openai
!pip install --upgrade langchain


In [154]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
result = model.invoke("Hello, whats 9 * 2  +2")

#print(result)   # it preints whole response with metadata and other information 

print(result.content)

### Types of messages in LangChain

- 1. *SystemMessage* : It's setup a Context for the conversation.
- 2. *HumanMessage*  : Prompts that user sents/writes.
- 3. *AiMessage*     : Ai responses

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

chat = ChatOpenAI(model_name="gpt-4", temperature=0.7)  ## temparature higher means creativity of the response would be less 

messages = [
    SystemMessage(content="You are a helpful assistant specializing in providing coding advice."),
    HumanMessage(content="Can you help me write a Python function to calculate factorial?"),
    AIMessage(content="Of course! Here's an example of a function to calculate the factorial of a number:\n"
                      "```python\n"
                      "def factorial(n):\n"
                      "    if n == 0 or n == 1:\n"
                      "        return 1\n"
                      "    else:\n"
                      "        return n * factorial(n - 1)\n"
                      "```")
]

messages.append(HumanMessage(content="Can you help me prime number code?"))

response = chat(messages)

messages.append(AIMessage(content=response.content))

for msg in messages:
    role = "System" if isinstance(msg, SystemMessage) else "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")


## Models

In [ ]:
!pip install langchain transformers huggingface_hub #Hugging face

!pip install -qU langchain-google-genai

In [ ]:
# gemini-1.5-pro
'''
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
'''

# hugging face

'''
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)'''

## Prompts 

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI

llm = ChatOpenAI(model= "gpt-4o", max_tokens=100)


human_message_template = "Write a {genre} story about a  {character} who is in {location}."

chat_prompt = ChatPromptTemplate.from_messages([("human", human_message_template)])

genre = "comedy"
character = "joker"
location = "city"

formatted_prompt = chat_prompt.format_messages(genre=genre, character=character, location=location)

response = llm.invoke(formatted_prompt)

print(response.content)


**Lets make usecase which is generating Language specific story based on prompt enginnering**

In [ ]:
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o", max_tokens=100)

# Define the prompt with placeholders for system and human messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative storyteller in {Language} Language."),
    ("human", "Write a {genre} story about a {character}.")
])

formatted_prompt = chat_prompt.format_messages(Language="Gujarati", genre="mystery", character="detective")

response = llm.invoke(formatted_prompt)

print(response)


## Chains 

*Type of chains*
- Sequencial Chain  : linear sequences of taks one after another 
- parallel chain    : doing multiple task at same time 
- Conditional chain : next set of task triggered based on condition (i.e. If user select FAQ so app directed him towards FaQ chainif he selects Billingg then related task chain triggerd )

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
llm = ChatOpenAI(model="gpt-4o", max_tokens=17)

prompt_template = "Tell me a about {name} "
prompt = PromptTemplate(
    input_variables=["name"], template=prompt_template
)
chain = prompt | llm | StrOutputParser()

chain.invoke("mahuva")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

# Input text
input_text = "In today's world, artificial intelligence is revolutionizing industries like healthcare, finance, and education."

# Initialize the model
model = ChatOpenAI(model="gpt-4o", max_tokens=120)

# Define chains
preprocess_chain = LLMChain(
    llm=model, 
    prompt=ChatPromptTemplate.from_messages([
        ("user", "Clean the following text by removing any special characters and extra spaces: {text}")
    ])
)
summarize_chain = LLMChain(
    llm=model, 
    prompt=ChatPromptTemplate.from_messages([
        ("user", "Summarize the following text in one paragraph: {text}")
    ])
)
translate_chain = LLMChain(
    llm=model, 
    prompt=ChatPromptTemplate.from_messages([
        ("user", "Translate the following text into French: {text}")
    ])
)

# Function to run a chain
def run_chain(chain, text):
    return chain.invoke({"text": text})["text"]

# Run all chains in parallel
def run_all_chains(input_text):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(
            lambda task: run_chain(task[0], task[1]),
            [(preprocess_chain, input_text), 
             (summarize_chain, input_text), 
             (translate_chain, input_text)]
        ))
    return {"preprocessed": results[0], "summarized": results[1], "translated": results[2]}

# Execute and display results
results = run_all_chains(input_text)
print("Preprocessed Text:", results["preprocessed"])
print("Summarized Text:", results["summarized"])
print("Translated Text:", results["translated"])


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

model = ChatOpenAI(model="gpt-4o", max_tokens=10)

sentiment_prompt = ChatPromptTemplate.from_messages([
    ("user", "Analyze the sentiment of the following feedback as Positive, Negative, or Neutral: {text}")
])
sentiment_chain = LLMChain(llm=model, prompt=sentiment_prompt)

# Positive Feedback Response Chain
positive_feedback_prompt = ChatPromptTemplate.from_messages([
    ("user", "Thank the user for their positive feedback and assure them of continued excellence: {text}")
])
positive_feedback_chain = LLMChain(llm=model, prompt=positive_feedback_prompt)

# Negative Feedback Response Chain
negative_feedback_prompt = ChatPromptTemplate.from_messages([
    ("user", "Apologize for the user's experience and ask for more details to resolve the issue: {text}")
])
negative_feedback_chain = LLMChain(llm=model, prompt=negative_feedback_prompt)

# Neutral Feedback Response Chain
neutral_feedback_prompt = ChatPromptTemplate.from_messages([
    ("user", "Thank the user for their feedback and encourage them to share more insights in the future: {text}")
])
neutral_feedback_chain = LLMChain(llm=model, prompt=neutral_feedback_prompt)

# Function to run conditional chaining
def respond_to_feedback(feedback_text):
    # Step 1: Analyze Sentiment
    sentiment_result = sentiment_chain.invoke({"text": feedback_text})["text"].strip().lower()
    
    if "positive" in sentiment_result:
        response = positive_feedback_chain.invoke({"text": feedback_text})["text"]
    elif "negative" in sentiment_result:
        response = negative_feedback_chain.invoke({"text": feedback_text})["text"]
    else:
        response = neutral_feedback_chain.invoke({"text": feedback_text})["text"]
    
    return response

feedback = "The service was amazing, but I wish the delivery was faster."

response = respond_to_feedback(feedback)
print("Feedback Response:", response)


## RAGs (retrieval-augmented generation)

- It just give LLMs additional knowledge



### Understanding RAG (Retrieval-Augmented Generation)

- **Challenge with Full-Text Input**:
  - Passing entire documents to LLMs is costly and slow due to high token usage.

- **Chunking the Data**:
  - Split text into smaller, meaningful chunks.
  - Ensure each chunk retains context and fits within token limits.

- **Using a Vector Database**:
  - Convert chunks into embeddings using models like Sentence Transformers.
  - Store embeddings in a vector database for efficient similarity search.

- **Query-Based Retrieval**:
  - When a question is asked, retrieve relevant chunks from the vector database.
  - Pass only the retrieved chunks to the LLM.


![RAGs Process](./Process.png)


### Contextual Embedding in Language Models

**Contextual embeddings** refer to the way words (or tokens) are represented in a high-dimensional vector space, where their meanings are influenced by the surrounding context in a given sentence or passage. Unlike traditional static word embeddings, which assign the same vector to a word regardless of its usage, **contextual embeddings** adapt to the specific context in which a word appears.

For example, consider the word **"Apple"** in the following two sentences:

1. **"What is Apple's revenue?"**
2. **"What is Apple's calorie?"**

In the first sentence, **"Apple"** refers to the **tech company** (Apple Inc.), and its embedding will reflect concepts like business, technology, and finance. In the second sentence, **"Apple"** refers to the **fruit**, and its embedding will be shaped by concepts like food, nutrition, and health.

This **context-dependent nature** of embeddings is a key feature of transformer-based models like **GPT-4**. The model generates different vector representations for the same word depending on its surrounding words, allowing it to understand and capture multiple meanings of the same token.


Vector embeddings
=================

Learn how to turn text into numbers, unlocking use cases like search.

**New embedding models**

`text-embedding-3-small` and `text-embedding-3-large`, our newest and most performant embedding models are now available, with lower costs, higher multilingual performance, and new parameters to control the overall size.

What are embeddings?
--------------------

OpenAI’s text embeddings measure the relatedness of text strings. Embeddings are commonly used for:

*   **Search** (where results are ranked by relevance to a query string)
*   **Clustering** (where text strings are grouped by similarity)
*   **Recommendations** (where items with related text strings are recommended)
*   **Anomaly detection** (where outliers with little relatedness are identified)
*   **Diversity measurement** (where similarity distributions are analyzed)
*   **Classification** (where text strings are classified by their most similar label)

An embedding is a vector (list) of floating point numbers. The [distance](#which-distance-function-should-i-use) between two vectors measures their relatedness. Small distances suggest high relatedness and large distances suggest low relatedness.

Visit our [pricing page](https://openai.com/api/pricing/) to learn about Embeddings pricing. Requests are billed based on the number of [tokens](/tokenizer) in the [input](/docs/api-reference/embeddings/create#embeddings/create-input).

How to get embeddings
---------------------

To get an embedding, send your text string to the [embeddings API endpoint](/docs/api-reference/embeddings) along with the embedding model name (e.g. `text-embedding-3-small`). The response will contain an embedding (list of floating point numbers), which you can extract, save in a vector database, and use for many different use cases:

Example: Getting embeddings

```javascript
import OpenAI from "openai";
const openai = new OpenAI();

const embedding = await openai.embeddings.create({
  model: "text-embedding-3-small",
  input: "Your text string goes here",
  encoding_format: "float",
});

console.log(embedding);
```

```python
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)
```

```bash
curl https://api.openai.com/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "input": "Your text string goes here",
    "model": "text-embedding-3-small"
  }'
```

The response will contain the embedding vector along with some additional metadata.

```json
{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.006929283495992422,
        -0.005336422007530928,
        -4.547132266452536e-05,
        -0.024047505110502243
      ],
    }
  ],
  "model": "text-embedding-3-small",
  "usage": {
    "prompt_tokens": 5,
    "total_tokens": 5
  }
}
```

By default, the length of the embedding vector will be 1536 for `text-embedding-3-small` or 3072 for `text-embedding-3-large`. You can reduce the dimensions of the embedding by passing in the [dimensions parameter](/docs/api-reference/embeddings/create#embeddings-create-dimensions) without the embedding losing its concept-representing properties. We go into more detail on embedding dimensions in the [embedding use case section](#use-cases).

Embedding models
----------------

OpenAI offers two powerful third-generation embedding model (denoted by `-3` in the model ID). You can read the embedding v3 [announcement blog post](https://openai.com/blog/new-embedding-models-and-api-updates) for more details.

Usage is priced per input token, below is an example of pricing pages of text per US dollar (assuming ~800 tokens per page):

|Model|~ Pages per dollar|Performance on MTEB eval|Max input|
|---|---|---|---|
|text-embedding-3-small|62,500|62.3%|8191|
|text-embedding-3-large|9,615|64.6%|8191|
|text-embedding-ada-002|12,500|61.0%|8191|

Use cases
---------

Here we show some representative use cases. We will use the [Amazon fine-food reviews dataset](https://www.kaggle.com/snap/amazon-fine-food-reviews) for the following examples.

### Obtaining the embeddings

The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text). For example:

|Product Id|User Id|Score|Summary|Text|
|---|---|---|---|---|
|B001E4KFG0|A3SGXH7AUHU8GW|5|Good Quality Dog Food|I have bought several of the Vitality canned...|
|B00813GRG4|A1D87F6ZCVE5NK|1|Not as Advertised|Product arrived labeled as Jumbo Salted Peanut...|

We will combine the review summary and review text into a single combined text. The model will encode this combined text and output a single vector embedding.


[Watch the video here on Embedding and Vectordatabase](https://youtu.be/ySus5ZS0b94?si=Heq5WFdpx2QdaqXI)


In [164]:
#### RAG example 


##  Agents

An **agent** in LangChain is an intelligent system that can make decisions about what actions to take based on a user’s input, using **tools** and potentially interacting with **external systems** to fulfill a task. Agents essentially act as **task managers** that coordinate the workflow between various components (like tools) and the model.

### Key Roles of an Agent:
- **Decision Making**: An agent interprets the user’s query and decides what actions are necessary to fulfill the request. This includes selecting the appropriate tool, querying a database, making an API call, or triggering some internal process.
  
- **Tool Management**: Agents can interact with multiple tools. They can invoke tools in a sequence or select one based on the specific task that needs to be performed. For example, if a user asks for weather information, an agent might call a weather API tool.

- **Task Sequencing**: Agents can manage complex workflows that require multiple steps. For example, booking a flight might involve checking availability, calculating the price, and then booking the flight—all of which the agent can handle in sequence.

### Example Agent Workflow:
1. **User Input**: "Book me a flight to Paris next Monday."
2. **Agent**:
   - **Interprets Intent**: Understands that the user is asking to book a flight.
   - **Selects Tools**: Chooses relevant tools for:
     - **Flight search** (API to check flight availability)
     - **Price calculation** (API to check prices)
     - **Booking** (system to complete the flight booking)
3. **Response**: The agent completes the task, potentially providing the user with a confirmation or next steps.

##  Tools

A **tool** in LangChain refers to an external resource, service, or function that an agent can call to perform specific tasks. Tools provide specialized capabilities that are not natively part of the LLM’s training, like accessing databases, querying APIs, interacting with file systems, or even running complex algorithms.

### Key Types of Tools:
- **APIs**: Tools that make external API calls to retrieve or send information. For example, an API that provides weather data or currency conversion.
  
- **Databases**: Tools that interact with databases to retrieve or store information. For example, a tool that queries a database for a user’s order history or fetches product details.

- **Calculations**: Tools that perform specific calculations or algorithms, such as running complex statistical models or doing price estimation.

- **Custom Functions**: Tools can also be custom-built functions that perform tasks such as sending emails, processing data, or accessing local files.

### Example Tools:
1. **Weather API Tool**: A tool that can fetch the weather for a specific city.
2. **Search Engine Tool**: A tool that can search the web for relevant articles or news.
3. **Database Query Tool**: A tool that queries a database to fetch information about users, products, etc.

### How Tools Are Used:
- When a user makes a request, the agent interprets the request and identifies the necessary tools.
- The agent calls the appropriate tool(s), receives the response, and integrates that response into the final output for the user.

## Example of Agents and Tools Working Together:

### Example 1: Booking a Flight
- **User Request**: "Book a flight from New York to London next week."
- **Agent**: 
  - The agent decides that it needs to check flight availability (tool: **Flight API**).
  - The agent queries the **Flight API** to get available flights.
  - The agent uses another tool to calculate the **price** for the flight (tool: **Price Calculation API**).
  - The agent then books the flight using the booking tool (tool: **Flight Booking API**).
  
  The agent combines the responses from each tool to present the user with a confirmed flight.

### Example 2: Weather Forecast
- **User Request**: "What’s the weather like in Tokyo tomorrow?"
- **Agent**: 
  - The agent calls the **Weather API Tool** to get tomorrow's weather in Tokyo.
  - The agent processes the response (e.g., "sunny with a high of 25°C") and presents it to the user.




In [165]:
# Agents 

In [166]:
# Tool building 

In [167]:
# agent and tool example 

# Memory 

**Memory** in LangChain refers to the ability of a language model (LLM) to retain and recall previous interactions, data, or context over multiple steps in a conversation or task. This is crucial in many real-world applications where the model needs to maintain consistency, track conversation history, or store intermediate results for future reference.

LangChain provides different types of memory mechanisms that can help LLMs remember previous interactions or relevant data. This allows the agent to perform more dynamic, context-aware tasks without having to repeat the entire process or re-examine all previous data.

## Key Concepts of Memory in LangChain

### 1. Persistent Memory
   - This type of memory stores information across multiple interactions, even if the LLM or agent is restarted. This is useful for applications like chatbots, virtual assistants, or systems where context needs to be maintained over time.
   - **Example**: A chatbot that remembers a user’s preferences or a customer support agent that tracks open tickets.

### 2. Short-term Memory
   - Short-term memory is typically used to hold the context within a single session or task. It helps the agent remember relevant information as long as the task is ongoing but doesn't persist once the session ends.
   - **Example**: In a task like booking a flight, the agent might remember the departure city, destination, and travel dates throughout the interaction but forget them after the task is completed.

### 3. Memory Types in LangChain:
   - **ConversationBufferMemory**: A simple memory that stores a buffer of conversation history, used for handling conversational context.
   - **VectorStoreMemory**: Stores information as vectors (embeddings), which is useful when dealing with large datasets or when performing tasks like semantic search.
   - **BufferMemory**: Stores a series of interactions or events in a queue-like fashion, useful for tracking a sequence of actions or operations.

### 4. Memory in Agents and Tools:
   - Agents can use memory to store the state of an ongoing task. For example, if an agent is processing a multi-step workflow (like booking a flight), memory can store intermediate results such as available flights, user preferences, or completed steps.
   - Tools can be designed to interact with memory to retrieve or update information during an agent's decision-making process.

## Benefits of Memory in LangChain
- **Contextual Understanding**: Memory allows an agent to maintain continuity in conversations or tasks, which improves the user experience.
- **Efficiency**: By remembering past information, the agent can reduce redundant queries or actions, leading to more efficient processing.
- **Flexibility**: Memory enables more complex and dynamic workflows by allowing the agent to adapt to the situation based on previously stored information.
